This file is for running pyscenic and preprocessing 10X data.

[tutorial ref](https://www.nature.com/articles/s41596-020-0336-2)

In [1]:
import scanpy as sc
import loompy as lp
import numpy as np

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [18]:
adata = sc.read_10x_mtx('/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/SP064_All_025_3p/outs/filtered_feature_bc_matrix/')

In [19]:
adata2 = adata
adata2

AnnData object with n_obs × n_vars = 84039 × 55421
    var: 'gene_ids', 'feature_types'

In [20]:
# compute the number of genes per cell (computes ‘n_genes' column)
sc.pp.filter_cells(adata2, min_genes=0)

# mito and genes/counts cutsadata_sub = adata2[CB]
#adata_sub
mito_genes = adata2.var_names.str.startswith('mt-')

In [21]:
# for each cell compute fraction of counts in mito genes vs. all genes
adata2.obs['percent_mito'] = np.ravel(np.sum(adata2[:, mito_genes].X, axis=1)) / np.ravel(np.sum(adata2.X, axis=1))

# add the total counts per cell as observations-annotation to adata
adata2.obs['n_counts'] = np.ravel(adata2.X.sum(axis=1))

In [22]:
adata2

AnnData object with n_obs × n_vars = 84039 × 55421
    obs: 'n_genes', 'percent_mito', 'n_counts'
    var: 'gene_ids', 'feature_types'

In [23]:
adata2.obs

,n_genes,percent_mito,n_counts
AAACCCAAGGCTCTAT-1,1396,0.000407,2455.0
AAACCCAAGTGACACG-1,1927,0.000000,3518.0
AAACCCACAATGGCCC-1,1276,0.001789,2236.0
AAACCCACAGATCCAT-1,1256,0.001809,2211.0
AAACCCAGTCACTCTC-1,538,0.001316,760.0
...,...,...,...
TTTGTTGGTTCGTTCC-9,2041,0.000228,4390.0
TTTGTTGGTTGGCCTG-9,2035,0.000245,4084.0
TTTGTTGGTTGGTACT-9,443,0.003044,657.0
TTTGTTGTCGACATTG-9,981,0.002983,1676.0


In [24]:
CB_list= open("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/20220310_pyscenic_pre_data_processing_meta_Seurat_pyscenic_CB.txt")
lines = CB_list.read().splitlines()
CB = np.array(lines)
CB

array(['AAACCCAAGGCTCTAT-1', 'AAACCCAAGTGACACG-1', 'AAACCCACAATGGCCC-1',
       ..., 'TTTGTTGGTTGGTACT-9', 'TTTGTTGTCGACATTG-9',
       'TTTGTTGTCGCGCCAA-9'], dtype='<U18')

In [25]:
adata_sub = adata2[CB]
adata_sub

View of AnnData object with n_obs × n_vars = 82298 × 55421
    obs: 'n_genes', 'percent_mito', 'n_counts'
    var: 'gene_ids', 'feature_types'

In [26]:
#subset = nCount_RNA < 30000 & nCount_RNA > 500 & percent.mt < 5)
#sc.pp.filter_cells(adata_sub, min_genes=200)
sc.pp.filter_genes(adata_sub, min_cells=3)

#adata_sub = adata_sub[adata_sub.obs['n_counts'] > 500, :]
#adata_sub = adata_sub[adata_sub.obs['n_counts'] < 30000, :]
#adata_sub = adata_sub[adata_sub.obs['percent_mito'] < 0.05, :]

Trying to set attribute `.var` of view, copying.


In [29]:
adata_sub.var

,gene_ids,feature_types,n_cells
4933401J01Rik,ENSMUSG00000102693,Gene Expression,3
Xkr4,ENSMUSG00000051951,Gene Expression,50409
Gm18956,ENSMUSG00000102851,Gene Expression,99
Gm1992,ENSMUSG00000089699,Gene Expression,19672
Gm7341,ENSMUSG00000103147,Gene Expression,15
...,...,...,...
AC125149.3,ENSMUSG00000079800,Gene Expression,64
AC168977.1,ENSMUSG00000079808,Gene Expression,22
AC149090.1,ENSMUSG00000095041,Gene Expression,38388
CAAA01118383.1,ENSMUSG00000063897,Gene Expression,2687


In [27]:
row_attrs = {"Gene": np.array(adata_sub.var_names),}
row_attrs


{'Gene': array(['4933401J01Rik', 'Xkr4', 'Gm18956', ..., 'AC149090.1',
        'CAAA01118383.1', 'CAAA01147332.1'], dtype=object)}

In [30]:
col_attrs = {
"CellID": np.array(adata_sub.obs_names),
"nGene": np.array(np.sum(adata_sub.X.transpose()>0, axis=0)).flatten(),
"nUMI": np.array(np.sum(adata_sub.X.transpose(),axis=0)).flatten(),
}
col_attrs

{'CellID': array(['AAACCCAAGGCTCTAT-1', 'AAACCCAAGTGACACG-1', 'AAACCCACAATGGCCC-1',
        ..., 'TTTGTTGGTTGGTACT-9', 'TTTGTTGTCGACATTG-9',
        'TTTGTTGTCGCGCCAA-9'], dtype=object),
 'nGene': array([1396, 1927, 1276, ...,  443,  981,  493]),
 'nUMI': array([2455., 3518., 2236., ...,  657., 1676.,  699.], dtype=float32)}

In [31]:
lp.create("/data/rajewsky/home/skim/Microglia_Heppner/201912_10X_9sets/pyscenic/filtered.loom", adata_sub.X.transpose(),row_attrs,
col_attrs)